In [ ]:
from llm.LLMModel import *
from ner.llm_ner.prompt_techniques.pt_abstract import PromptTechnique
from ner.llm_ner.prompt_techniques.pt_discussion import PT_OutputList
from ner.llm_ner.prompt_techniques.pt_gpt_ner import PT_GPT_NER
from ner.llm_ner.prompt_techniques.pt_wrapper import PT_Wrapper
from ner.llm_ner.few_shots_techniques import *
from ner.llm_ner.ResultInstance import load_all_results, load_result
%reload_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nlt

In [ ]:

df_results = load_all_results()
df_results

In [ ]:
df_results.sort_values(['model', 'prompt_technique', 'nb_test_run'])

In [ ]:
df_results.groupby(['model', 'prompt_technique', 'nb_test_run']).head(1).sort_values('f1_mean', ascending = False)

In [15]:
df_results['model_base'] = df_results['model'].map(lambda s : '-'.join(s.split('-')[:-4]))
df_results['nb_train_samples'] = df_results['model'].map(lambda s : s.split('-')[-2])
df_results['finetuned'] = df_results['model'].map(lambda s : 'ft' in s)
df_results['nb_samples_tested'] = df_results['nb_test_run'] * df_results['len_data_test']

In [9]:
df_results

,f1_mean,f1_conf_inter,model,prompt_technique,few_shot_tecnique,nb_few_shots,precision,verifier,len_data_train,len_data_test,...,confidence_interval,distribution_used,precision_mean,precision_conf_inter,recall_mean,recall_conf_inter,model_base,nb_tr_samples,nb_train_samples,finetuned
13,0.923,"(0.877, 0.969)",mistral-7b-v0.1-ft-discussion-2000-Q5_0,discussion,entity,5,no-precision,None,1538,50,...,0.95,Student,0.919,"(0.831, 1.007)",0.873,"(0.800, 0.946)",mistral-7b-v0.1,mistral-7b-v0.1-ft,2000,True
12,0.912,"(0.836, 0.988)",mistral-7b-v0.1-ft-discussion-2000-Q5_0,discussion,sentence,5,yes,None,1538,50,...,0.95,Student,0.924,"(0.817, 1.031)",0.855,"(0.740, 0.970)",mistral-7b-v0.1,mistral-7b-v0.1-ft,2000,True
0,0.894,"(0.887, 0.902)",mistral-7b-v0.1-ft-wrapper-2000-Q5_0,wrapper,no-shots,5,True,None,1538,50,...,0.95,Student,0.855,"(0.846, 0.864)",0.634,"(0.620, 0.648)",mistral-7b-v0.1,mistral-7b-v0.1-ft,2000,True
9,0.890,"(0.846, 0.935)",mistral-7b-v0.1-ft-discussion-2000-Q5_0,discussion,entity,5,yes,None,1538,50,...,0.95,Student,0.875,"(0.744, 1.006)",0.821,"(0.757, 0.885)",mistral-7b-v0.1,mistral-7b-v0.1-ft,2000,True
10,0.890,"(0.845, 0.936)",mistral-7b-v0.1-ft-discussion-2000-Q5_0,discussion,random,5,yes,None,1538,50,...,0.95,Student,0.870,"(0.813, 0.927)",0.823,"(0.750, 0.895)",mistral-7b-v0.1,mistral-7b-v0.1-ft,2000,True
47,0.889,"(0.877, 0.902)",mistral-7b-v0.1-ft-wrapper-5000-Q5_0,wrapper,no-shots,5,True,None,1538,50,...,0.95,Student,0.819,"(0.790, 0.848)",0.662,"(0.641, 0.682)",mistral-7b-v0.1,mistral-7b-v0.1-ft,5000,True
16,0.888,"(0.840, 0.935)",mistral-7b-v0.1-ft-discussion-2000-Q5_0,discussion,sentence,5,no-precision,None,1538,50,...,0.95,Student,0.875,"(0.839, 0.911)",0.827,"(0.772, 0.883)",mistral-7b-v0.1,mistral-7b-v0.1-ft,2000,True
6,0.886,"(0.844, 0.928)",mistral-7b-v0.1-ft-discussion-2000-Q5_0,discussion,random,5,no-precision,None,1538,50,...,0.95,Student,0.865,"(0.822, 0.908)",0.825,"(0.770, 0.880)",mistral-7b-v0.1,mistral-7b-v0.1-ft,2000,True
5,0.881,"(0.872, 0.891)",mistral-7b-v0.1-ft-wrapper-2000-Q5_0,wrapper,sentence,5,True,None,1538,50,...,0.95,Student,0.821,"(0.803, 0.838)",0.623,"(0.608, 0.637)",mistral-7b-v0.1,mistral-7b-v0.1-ft,2000,True
17,0.881,"(0.874, 0.887)",mistral-7b-v0.1-ft-discussion-2000-Q5_0,discussion,random,5,True,None,1538,50,...,0.95,Student,0.872,"(0.862, 0.882)",0.816,"(0.807, 0.826)",mistral-7b-v0.1,mistral-7b-v0.1-ft,2000,True


In [40]:
ft_results = df_results[['f1_mean', 'f1_conf_inter', 'prompt_technique',	'few_shot_tecnique','nb_few_shots', 'nb_samples_tested', 'model_base', 'nb_train_samples', 'finetuned']]
ft_results = ft_results[ft_results['finetuned']]
ft_results = ft_results.sort_values(['prompt_technique',	'few_shot_tecnique', 'f1_mean'], ascending=False).dropna()

In [43]:
# ['f1_mean', 'f1_conf_inter', 'prompt_technique', 'few_shot_tecnique',
    #    'nb_few_shots', 'nb_samples_tested', 'model_base', 'nb_train_samples',
    #    'finetuned']
grouped = ft_results.groupby(['prompt_technique', 'few_shot_tecnique',
       'nb_few_shots', 'nb_samples_tested', 'model_base', 'nb_train_samples'])
grouped 

idx_max = grouped['f1_mean'].idxmax()

ft_cleaned = ft_results.loc[idx_max]

In [45]:
df = ft_cleaned[ft_cleaned['nb_samples_tested'] == 500]
# Create dummy columns for 'few_shot_tecnique'
df.pivot(index=['prompt_technique',
       'nb_few_shots', 'nb_samples_tested', 'model_base', 'nb_train_samples'], columns='few_shot_tecnique', values= 'f1_mean')

few_shot_tecnique                                                                 entity  \
prompt_technique nb_few_shots nb_samples_tested model_base      nb_train_samples           
discussion       5            500               mistral-7b-v0.1 10000              0.839   
                                                                2000               0.881   
wrapper          5            500               mistral-7b-v0.1 10000              0.652   
                                                                2000                 NaN   
                                                                5000                 NaN   

few_shot_tecnique                                                                 no-shots  \
prompt_technique nb_few_shots nb_samples_tested model_base      nb_train_samples             
discussion       5            500               mistral-7b-v0.1 10000                0.860   
                                                                2000                 0.722   
wrapper          5            500               mistral-7b-v0.1 10000                0.504   
                                                                2000                 0.894   
                                                                5000                 0.889   

few_shot_tecnique                                                                 random  \
prompt_technique nb_few_shots nb_samples_tested model_base      nb_train_samples           
discussion       5            500               mistral-7b-v0.1 10000              0.834   
                                                                2000               0.881   
wrapper          5            500               mistral-7b-v0.1 10000              0.588   
                                                                2000                 NaN   
                                                                5000                 NaN   

few_shot_tecnique                                                                 sentence  
prompt_technique nb_few_shots nb_samples_tested model_base      nb_train_samples            
discussion       5            500               mistral-7b-v0.1 10000                0.783  
                                                                2000                 0.879  
wrapper          5            500               mistral-7b-v0.1 10000                0.635  
                                                                2000                 0.881  
                                                                5000                 0.880

In [9]:
model  = "mistral-7b-v0.1-ft-discussion-10000-Q5_0"
result = load_result(model, PT_OutputList.name(), FST_Sentence.name(), with_precision = "10_False")

for r in result.res_insts :
    r.show_cm()
    r.analyse_results()

In [ ]:
r.f1

In [ ]:
from ner.Datasets.Conll2003Dataset import load_conll_dataset
data = load_conll_dataset(cleaned = True)
[(d['text'], d['spans']) for d in data if str(d['spans']) == """[['Switzerland', 'LOC'], ["Alfonse D'Amato", 'PER'], ['U.S. Senate Banking Committee', 'ORG'], ['Poland', 'LOC'], ['Polish', 'MISC'], ['Swiss', 'MISC'], ['Poland', 'LOC']]"""]

In [ ]:
[(d['text'], d['spans']) for d in data if 'Hansenne' in d['text']]